In [1]:
import pandas as pd
import numpy as np

In [2]:
#from matplotlib import pyplot as plt

# dataframe preparation

In [3]:
data=pd.read_csv('data/Dataset/Dataset/Training/Features_Variant_1.csv', header=None).drop_duplicates()

In [4]:
#get and fit features names
features_names=np.array(['page_likes_num',#1
               'page_checkins',#2
               'page_talking_about',#3
                'page_cat',#4
                'page_statistics',#5-29 #mean, avg etc.
                'comments_num_before_base_time',#30
                'comments_num_in_last_24_hours',#31 #last day
                'comments_num_in_last_48_to_24_hours',#32 #day before last
                'comments_num_in_first_24_hours',#33
                'comments_difference_in_last_two_days', #34 (32-31)
                'base_time', #35
                'character_num_in_post', #36
                'share_num',#37
                'post_promotion', #38 binary
                'h_local', #39 This describes the H hrs, for which we have the target variable/ comments received. 
                'post_published_weekday', #40-46 This represents the day(Sunday...Saturday) on which the post was published. 
                'base_ditetime_weekday', #47-53 This represents the day(Sunday...Saturday) on selected base Date/Time. 
                'target' #54 The no of comments in next H hrs(H is given in Feature no 39).                
               ])

for index in range(5,29):
    features_names=np.insert(features_names, index, features_names[4]+'_'+str(index-4))
    
weekday=('sunday', 'monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday')    

for index in range(40,47):
    features_names=np.insert(features_names,index, features_names[39]+'_'+ weekday[index-40])
features_names=np.delete(features_names, 39)

for index in range(47,54):
    features_names=np.insert(features_names,index, features_names[46]+'_'+ weekday[index-47])
features_names=np.delete(features_names, 46)

data.columns=features_names

In [5]:
y=data.nunique(axis=0).sort_values()
print(y.head())

post_promotion                      1
post_published_weekday_sunday       2
post_published_weekday_monday       2
post_published_weekday_tuesday      2
post_published_weekday_wednesday    2
dtype: int64


In [6]:
from sklearn.utils import shuffle
data=data.drop(columns=['post_promotion']) #post promotion has only zero values we can drop this col because it isn't informative
data=shuffle(data).reset_index(drop=True)

# data preparation

In [7]:
#'page_cat' to dummy because it is a category
data=pd.concat([data,pd.get_dummies(data['page_cat'], prefix='page_cat')], axis=1)
data=data.drop(columns=['page_cat'])

In [8]:
def normalize(df):
    std=np.std(df, ddof=1).replace(0, 1)
    mean=np.mean(df)
    return (df-mean)/std, mean, std

def normalize_with_params(df, mean, std):
    return (df-mean)/std

# learning

In [9]:
def custom_RMSE(target, pred, gamma, w):
    return np.sqrt(
        np.sum((target-pred)**2)/len(pred)+gamma*np.linalg.norm(w)
    )

def get_para_w(old_w, df, pred, lam, gamma, terms_num, iter_num):#, start_ind):
    indices= np.random.choice(df.shape[1]-1, terms_num, replace=False)
    #indices=np.array(range(start_ind, start_ind+terms_num))%df.shape[0]
    #start_ind=start_ind+terms_num
    old_w=old_w+2*lam*(
          (np.dot((df.loc[indices,'target']-pred[indices]).values,df.iloc[indices,:-1].values))/(len(pred))#*iter_num)
        )
    return old_w, indices#, start_ind

def get_para_w0(old_w0, df, pred, lam, gamma, indices, iter_num):
    return old_w0+2*lam*(
        (np.sum(df.loc[indices,'target']-pred[indices]))/(len(pred))#*iter_num)
        )
def get_prediction(w,w0,x_df):
    return np.dot(x_df, w)+w0

def RMSE_plot(RMSE_arr):
    plt.figure(figsize=(15,7))   
    plt.plot(RMSE_arr)
    plt.xlabel('iteration number')
    plt.ylabel('RMSE value')
    plt.title('algo scores')
    plt.show()
    
def gradient_descend(df, lam, gamma, terms_num, max_iter):
    df=df.reset_index(drop=True)
    w=np.array([0]*(df.shape[1]-1), dtype=float) #one column is the target
    w0=0.0
    prediction=get_prediction(w,w0,df.iloc[:,:-1])
    minRMSE=custom_RMSE(df['target'], prediction, gamma, w)
    #RMSE_arr=np.array([minRMSE], dtype=float)
    params=np.append(w,w0)
    best_params=params
    best_pred=prediction

    curr_err=10
    err=1e-5
    
    iter_num=0
    start_ind=0
    while (curr_err>err) & (iter_num<max_iter) :    
        iter_num+=1
        w, indices=get_para_w(w, df, prediction, lam, gamma, terms_num, iter_num)
        w0=get_para_w0(w0, df, prediction, lam, gamma, indices, iter_num)
        
        #first variant of stopping criterion:
        curr_err=prediction.copy()
        prediction=get_prediction(w,w0,df.iloc[:,:-1])
        curr_err=np.linalg.norm(curr_err-prediction)#||y(k+1)-y(k)||
        
        """
        #another stopping criterion:
        curr_err=params.copy() #old params        
        params=np.append(w,w0)
        curr_err=np.linalg.norm(curr_err-params)#||w(k+1)-w(k)||
        prediction=get_prediction(w,w0,df.iloc[:,:-1])
        """
        params=np.append(w,w0)
        #RMSE_arr=np.append(RMSE_arr,custom_RMSE(df['target'], prediction, gamma, w))
        if custom_RMSE(df['target'], prediction, gamma, w)<minRMSE:
            minRMSE=custom_RMSE(df['target'], prediction, gamma, w)
            best_params=params.copy()
            best_pred=prediction
            
    print('Iterations number is:' +str(iter_num))
    if iter_num==max_iter:
        print('The maximum number of iterations was reached.')
    print('RMSE is:'+str(minRMSE))
    print('R2 is:'+str(custom_R2(df.iloc[:,-1], best_pred)))
    
    #RMSE_plot(RMSE_arr)        
    return best_params

In [10]:
#set algo params:
lam=1e-3
gamma=1e-3
terms_num=10
max_iter=1e4

# Cross-val

In [11]:
def custom_R2(target, pred):
    return 1-(np.sum((target-pred)**2))/(np.sum((target-np.mean(target))**2))

In [12]:
res=pd.DataFrame()

parts=range(1,6)
idxs=[round(data.shape[0]/5)*(i-1) for i in range(1,6)]
idxs.append(data.shape[0])

for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])
    #features normalization (xi-mean)/std_err:
    train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    params=gradient_descend(train_part, lam, gamma, terms_num, max_iter) #model    
    test_pred=get_prediction(params[:-1],params[-1],test_part.iloc[:,:-1])
    RMSE=custom_RMSE(test_part['target'], test_pred, gamma, params[:-1])
    r2=custom_R2(test_part['target'], test_pred)
    params=np.insert(params, 0 , [RMSE,r2])
    res[part]=params

C:\Users\risam\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9855556011406843
R2 is:-0.6634463958122481
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9912864562828873
R2 is:-0.2611799953293832
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9913103090501844
R2 is:-0.2124246487290029
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9916210820160788
R2 is:-0.20060304618993996
Iterations number is:10000
The maximum number of iterations was reached.
RMSE is:0.9810837530757648
R2 is:-1.8330992864925597


In [13]:
text_idxs=np.array(['w'+str(i) for i in range(1,res.shape[0]-2)])
text_idxs=np.append(text_idxs,'w0')
text_idxs=np.insert(text_idxs, 0, ['RMSE', 'r2'])
res.index=text_idxs

res['mean']=np.mean(res, axis=1)
res['std']=np.std(res.iloc[:,:-1], ddof=1, axis=1)

In [14]:
res

,1,2,3,4,5,mean,std
RMSE,1.038880,1.076031,1.157678,0.936038,3.943480,1.630421,1.295489e+00
r2,0.025175,0.021039,0.018485,0.019610,-28.136325,-5.610403,1.259237e+01
w1,0.000468,0.000443,0.000429,0.000498,0.000380,0.000444,4.417665e-05
w2,0.001337,-0.000093,-0.000087,-0.000099,-0.000098,0.000192,6.398423e-04
w3,0.001185,0.001679,0.001631,0.001674,0.001449,0.001523,2.113194e-04
w4,0.000036,0.000159,0.000193,0.000212,0.002423,0.000605,1.018926e-03
w5,0.001253,0.000295,0.000202,0.000145,0.001191,0.000617,5.553483e-04
w6,0.001515,0.000277,0.000223,0.000203,0.003180,0.001079,1.298710e-03
w7,0.001418,0.000266,0.000236,0.000234,0.003794,0.001190,1.542124e-03
w8,0.001503,0.000309,0.000230,0.000190,0.001963,0.000839,8.332069e-04


In [15]:
#this part is for comparison
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
#this part is for unnormalized data, next cell contains for normalized data
model=LinearRegression()
scores = cross_val_score(model, data.iloc[:,:-1], data.iloc[:,-1], cv=5, scoring='r2')
print('R2 scores for each fold is:')
print(scores)

R2 scores for each fold is:
[1.         1.         0.88876662 0.88876662 1.        ]


In [16]:
regres=np.array([])
data=data.reset_index(drop=True)
for part in parts:
    #separate dataset
    test_part=data[idxs[part-1]:idxs[part]]
    train_part=data.drop(data.index[idxs[part-1]:idxs[part]])
    #features normalization (xi-mean)/std_err:
    train_part.iloc[:,:-1], mean, std=normalize(train_part.iloc[:,:-1])#fit and transform train
    test_part.iloc[:,:-1]=normalize_with_params(test_part.iloc[:,:-1], mean, std) #transform test
    model=LinearRegression().fit(train_part.iloc[:,:-1], train_part.iloc[:,-1])    
    test_pred=model.predict(test_part.iloc[:,:-1])    
    r2=custom_R2(test_part['target'], test_pred)
    regres=np.append(regres, r2)
print('R2 for LinReg on 5 folds:')
print(regres)

C:\Users\risam\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


R2 for LinReg on 5 folds:
[-0.0021709  -0.0023462  -0.00205638 -0.00215451 -0.00436781]
